In [ ]:
# If PEPit is not installed yet, you can run this cell.
!pip install pepit

In [1]:
# Run this code before executing the cell below
from math import sqrt
import numpy as np
from PEPit import PEP
from PEPit.function import Function


class LipschitzNegativeComonotoneOperator(Function):
    
    def __init__(self,
                 rho,
                 L=1.,
                 is_leaf=True,
                 decomposition_dict=None,
                 reuse_gradient=True):
        
        super().__init__(is_leaf=is_leaf,
                         decomposition_dict=decomposition_dict,
                         reuse_gradient=True)
        # Store L and rho
        self.rho = rho
        self.L = L

        if self.L == np.inf:
            print("\033[96m(PEPit) The class of Lipschitz negative comonotone operators is necessarily continuous.\n"
                  "To instantiate an operator, please avoid using the class LipschitzNegativeComonotoneOperator with\n"
                  " L == np.inf. Instead, please use the class NegativeComonotoneOperator (which accounts for the fact\n"
                  "that the image of the operator at certain points might not be a singleton).\033[0m")

    def add_class_constraints(self):
        for point_i in self.list_of_points:

            xi, gi, fi = point_i

            for point_j in self.list_of_points:

                xj, gj, fj = point_j

                if (xi != xj) | (gi != gj):
                    # Interpolation conditions of negative comonotone operator class
                    self.add_constraint((gi - gj) * (xi - xj) + self.rho * (gi - gj)**2 >= 0)
                    # Interpolation conditions of Lipschitz operator class
                    self.add_constraint((gi - gj)**2 - self.L**2 * (xi - xj)**2 <= 0)

class LipschitzStarNegativeComonotoneOperator(Function):
    
    def __init__(self,
                 rho,
                 L=1.,
                 is_leaf=True,
                 decomposition_dict=None,
                 reuse_gradient=True):
        
        super().__init__(is_leaf=is_leaf,
                         decomposition_dict=decomposition_dict,
                         reuse_gradient=True)
        # Store L and rho
        self.rho = rho
        self.L = L

        if self.L == np.inf:
            print("\033[96m(PEPit) The class of Lipschitz star negative comonotone operators is necessarily continuous.\n"
                  "To instantiate an operator, please avoid using the class LipschitzNegativeComonotoneOperator with\n"
                  " L == np.inf. Instead, please use the class NegativeComonotoneOperator (which accounts for the fact\n"
                  "that the image of the operator at certain points might not be a singleton).\033[0m")

    def add_class_constraints(self):
        for point_i in self.list_of_points:

            xi, gi, fi = point_i

            for point_j in self.list_of_points:

                xj, gj, fj = point_j

                if (xi != xj) | (gi != gj):
                    # Interpolation conditions of Lipschitz operator class
                    self.add_constraint((gi - gj)**2 - self.L**2 * (xi - xj)**2 <= 0)
                    
        for point_i in self.list_of_stationary_points:

            xi, gi, fi = point_i

            for point_j in self.list_of_points:

                xj, gj, fj = point_j

                if point_i != point_j:
                    # Interpolation conditions of star negative comonotone operator class
                    self.add_constraint(gj * (xj-xi) + self.rho *  gj ** 2 >= 0)

## $\rho$-star negative comonotone and Lipschitz operator

In [2]:
def wc_optimisticgradient_star(n, gamma1, gamma2, L, rho, verbose=1):

    # Instantiate PEP
    problem = PEP()

    # Declare an indicator function and a monotone operator
    F = problem.declare_function(LipschitzStarNegativeComonotoneOperator, rho=rho, L=L)


    # Start by defining its unique optimal point xs = x_*
    xs = F.stationary_point()

    # Then define the starting point x0 of the algorithm and its gradient value g0
    x0 = problem.set_initial_point()

    # Set the initial constraint that is the distance between x0 and x^*
    problem.set_initial_condition((x0 - xs) ** 2 <= 1)

    # Compute n steps of the optimistic gradient method starting from x0
    x = x0
    xtilde = x
    V = F.gradient(x)
    obj = V**2
    for _ in range(n):
        xtilde = x - gamma1 * V
        V = F.gradient(xtilde)
        x = x - gamma2 * V
        obj = obj + F.gradient(x)**2

    # Set the performance metric
    problem.set_performance_metric(obj/(n+1))

    # Solve the PEP
    pepit_verbose = max(verbose, 0)
    pepit_tau = problem.solve(verbose=pepit_verbose)

    # Compute theoretical guarantee (for comparison)
    theoretical_tau = 1/(gamma1*gamma2*(1-L**2*(gamma1+gamma2)**2)*(n+1))

    # Print conclusion if required
    if verbose != -1:
        print('*** Example file: worst-case performance of the optimistic gradient method ***')
        print('\tPEPit guarantee:\t 1/(n+1) \sum(k=0 to n) ||F(x(k))||^2 <= {:.6} ||x0 - xs||^2'.format(pepit_tau))
        print('\tTheorem 4.2:\t 1/(n+1) \sum(k=0 to n) ||F(x(k))||^2 <= {:.6} ||x0 - xs||^2'.format(theoretical_tau))

    # Return the worst-case guarantee of the evaluated method ( and the reference theoretical value)
    return pepit_tau, theoretical_tau

In [3]:
n = 30


L = 1
rho = .1              # should satisfy 0<= rho < 1/2/L
gamma1 = 1/2          # should satisfy 2*rho < gamma1 < 1/L
gamma2 = .2           # should satisfy 0 < gamma2 <= min(1/L-gamma1, gamma1-2*rho)

wc_optimisticgradient_star(n, gamma1, gamma2, L, rho, verbose=1)

(PEPit) Setting up the problem: size of the main PSD matrix: 63x63
(PEPit) Setting up the problem: performance measure is minimum of 1 element(s)
(PEPit) Setting up the problem: initial conditions and general constraints (1 constraint(s) added)
(PEPit) Setting up the problem: interpolation conditions for 1 function(s)
		 function 1 : 3843 constraint(s) added
(PEPit) Setting up the problem: 0 lmi constraint(s) added
(PEPit) Compiling SDP
(PEPit) Calling SDP solver
(PEPit) Solver status: optimal (solver: MOSEK); optimal value: 0.3788572332601821
(PEPit) Postprocessing: solver's output is not entirely feasible (smallest eigenvalue of the Gram matrix is: -4.9e-09 < 0).
 Small deviation from 0 may simply be due to numerical error. Big ones should be deeply investigated.
 In any case, from now the provided values of parameters are based on the projection of the Gram matrix onto the cone of symmetric semi-definite matrix.
*** Example file: worst-case performance of the optimistic gradient met

(0.3788572332601821, 0.6325110689437065)

## $\rho$-negative comonotone and Lipschitz operator

In [4]:
def wc_optimisticgradient(n, gamma, L, rho, verbose=1):

    # Instantiate PEP
    problem = PEP()

    # Declare an indicator function and a monotone operator
    F = problem.declare_function(LipschitzNegativeComonotoneOperator, rho=rho, L=L)


    # Start by defining its unique optimal point xs = x_*
    xs = F.stationary_point()

    # Then define the starting point x0 of the algorithm and its gradient value g0
    x0 = problem.set_initial_point()

    # Set the initial constraint that is the distance between x0 and x^*
    problem.set_initial_condition((x0 - xs) ** 2 <= 1)

    # Compute n steps of the optimistic gradient method starting from x0
    x = x0
    xtilde = x
    V = F.gradient(x)
    obj = V**2
    for _ in range(n):
        xtilde = x - gamma * V
        V = F.gradient(xtilde)
        x = x - gamma * V

    # Set the performance metric
    obj = F.gradient(x)**2
    problem.set_performance_metric(obj)

    # Solve the PEP
    pepit_verbose = max(verbose, 0)
    pepit_tau = problem.solve(verbose=pepit_verbose)

    # Compute theoretical guarantee (for comparison)
    theoretical_tau = 717/(n*gamma*(gamma-3*rho)+800*gamma**2)

    # Print conclusion if required
    if verbose != -1:
        print('*** Example file: worst-case performance of the optimistic gradient method ***')
        print('\tPEPit guarantee:\t ||F(x(n))||^2 <= {:.6} ||x0 - xs||^2'.format(pepit_tau))
        print('\tTheorem 4.2:\t ||F(x(n))||^2 <= {:.6} ||x0 - xs||^2'.format(theoretical_tau))

    # Return the worst-case guarantee of the evaluated method ( and the reference theoretical value)
    return pepit_tau, theoretical_tau

In [5]:
n = 10


L = 1
rho = .05             # should satisfy 0<= rho < 5/62/L
gamma = 10/31/L       # should satisfy 4*rho <= gamma <= 10/31/L

wc_optimisticgradient(n, gamma, L, rho, verbose=1)

(PEPit) Setting up the problem: size of the main PSD matrix: 14x14
(PEPit) Setting up the problem: performance measure is minimum of 1 element(s)
(PEPit) Setting up the problem: initial conditions and general constraints (1 constraint(s) added)
(PEPit) Setting up the problem: interpolation conditions for 1 function(s)
		 function 1 : 312 constraint(s) added
(PEPit) Setting up the problem: 0 lmi constraint(s) added
(PEPit) Compiling SDP
(PEPit) Calling SDP solver
(PEPit) Solver status: optimal (solver: MOSEK); optimal value: 0.4733421380779232
(PEPit) Postprocessing: solver's output is not entirely feasible (smallest eigenvalue of the Gram matrix is: -2.33e-09 < 0).
 Small deviation from 0 may simply be due to numerical error. Big ones should be deeply investigated.
 In any case, from now the provided values of parameters are based on the projection of the Gram matrix onto the cone of symmetric semi-definite matrix.
*** Example file: worst-case performance of the optimistic gradient met

(0.4733421380779232, 8.555745948966289)